In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from matplotlib.lines import Line2D
from mpl_toolkits.mplot3d import axes3d, Axes3D
from mpl_toolkits import mplot3d

In [2]:
def load_dataset1(mu1,mu2,mu3):

    cov1 = np.array([[1.5,0],[0,1.5]])
    cov2 = np.array([[2,0],[0,2]])
    cov3 = np.array([[1,0],[0,1]])
    covariances = [None] * 3
    covariances[0] = cov1
    covariances[1] = cov2
    covariances[2] = cov3

    x1 = np.array(np.random.multivariate_normal(mu1,cov1,500))
    x2 = np.array(np.random.multivariate_normal(mu2,cov2,500))
    x3 = np.array(np.random.multivariate_normal(mu3,cov3,500))

    return covariances,x1,x2,x3


def load_dataset2(mu1,mu2,mu3):

    cov1 = np.array([[1.5,0.1],[0.1,0.5]])
    cov2 = np.array([[1,-0.20],[-0.20,2]])
    cov3 = np.array([[2,-0.25],[-0.25,1.5]])
    covariances = [None] * 3
    covariances[0] = cov1
    covariances[1] = cov2
    covariances[2] = cov3

    x1 = np.array(np.random.multivariate_normal(mu1,cov1,500))
    x2 = np.array(np.random.multivariate_normal(mu2,cov2,500))
    x3 = np.array(np.random.multivariate_normal(mu3,cov3,500))

    return covariances,x1,x2,x3


def get_prior(y):
    y_unique = np.unique(y)
    len_classes = len(y_unique)
    priors = [None] * len_classes
    for i in range(len_classes):
        label = y_unique[i]
        mask = (y == label)
        len_samples_in_class = np.sum(mask)
        prior_of_class = len_samples_in_class / len(y)
        priors[i] = prior_of_class
    return(priors)


def classify(X,sigma,Mu,Phi):
        estimated_class = np.zeros((X.shape[0]))
        for i in range(X.shape[0]):
            P_y = np.zeros((3))
            for j in range(3):
                const = 1./(((2*np.pi)**(1.5))*(np.linalg.det(sigma[j])**(0.5)))
                likelihood  = np.exp(-1/2 * (X[i] - Mu[j]).T @ np.linalg.inv(sigma[j]) @ (X[i] - Mu[j]))
                P_y[j]=const*likelihood*Phi[j]

            estimated_class[i] = np.argmax(P_y)+1
        return estimated_class


def score(preds,y):
    score = (preds == y).mean()
    return score*100


def plot2d(x,Mu,sigma,y,phi,dataset):
    fig2 = plt.figure()
    ax2 = fig2.add_subplot(1,1,1)
    ax2.set_xlabel("x1")
    ax2.set_ylabel("x2")
    fig2.suptitle('2D estimated PDFs and decision boundary')
    color = ['blue','red','green']

    f0 = np.linspace(x[:,0].min(),x[:,0].max())
    f1 = np.linspace(x[:,1].min(),x[:,1].max())
    X, Y = np.meshgrid(f0,f1)

    for c in range(3):
        def pdf(point):
            const = 1./(((2*np.pi)**(1.5))*(np.linalg.det(sigma[c])**(0.5)))
            return const*np.exp(-1/2.*(point-Mu[c]).T @ np.linalg.inv(sigma[c]) @ (point-Mu[c]))
        z = np.array([pdf(np.array(ponit)) for ponit in zip(np.ravel(X),np.ravel(Y))])
        Z = z.reshape(X.shape)

        ax2.contour(X, Y, Z,colors=color[c])
        plot_line(X, y,sigma,Mu,phi,dataset,ax2)




def plot3d(x,Mu,sigma):

    fig3 = plt.figure(figsize=(6,6))
    ax3D = plt.axes(projection='3d')
    fig3.suptitle('3D estimated PDF')
    ax3D.view_init(10,-100)

    ax3D.set_xlabel("x1")
    ax3D.set_ylabel("x2")
    ax3D.set_zlabel("P(X|y)")
    color = ['blue','red','green']

    f0 = np.linspace(x[:,0].min(),x[:,0].max())
    f1 = np.linspace(x[:,1].min(),x[:,1].max())
    X, Y = np.meshgrid(f0,f1)

    for c in range(3):
        def pdf(point):
            const = 1./(((2*np.pi)**(1.5))*(np.linalg.det(sigma[c])**(0.5)))
            return const*np.exp(-1/2.*(point-Mu[c]).T @ np.linalg.inv(sigma[c]) @ (point-Mu[c]))
        zs = np.array([pdf(np.array(ponit)) for ponit in zip(np.ravel(X),np.ravel(Y))])
        Z = zs.reshape(X.shape)

        ax3D.plot_surface(X, Y,Z,alpha=.3,rstride=1,cstride=1,color=color[c],edgecolor='none')
        ax3D.contour3D(X, Y, Z,60, colors=color[c])


def plot_scot(X,y,pred,sigma,mu,phi,dataset,titl):
    fig1 = plt.figure(figsize=(6,6))
    ax1 = fig1.add_subplot(1,1,1)
    xone = X[:,0]
    xtwo = X[:,1]
    for i in range(len(y)):
        if(y[i] == pred[i] and y[i] == 1 ):
            clas1 = ax1.scatter(xone[i], xtwo[i], alpha = .5, marker = 'o', color = 'b')
        if(y[i] == pred[i] and y[i] == 2 ):
            clas2 = ax1.scatter(xone[i], xtwo[i], alpha = .5, marker = 'o', color = 'r')
        if(y[i] == pred[i] and y[i] == 3 ):
            clas3 = ax1.scatter(xone[i], xtwo[i], alpha = .5, marker = 'o', color = 'g')

        if(y[i] != pred[i] and y[i] == 1 ):
            mis1 = ax1.scatter(xone[i], xtwo[i], marker = 'x', color = 'b')
        if(y[i] != pred[i] and y[i] == 2 ):
            mis2 = ax1.scatter(xone[i], xtwo[i], marker = 'x' , color = 'r')
        if(y[i] != pred[i] and y[i] == 3 ):
            mis3 = ax1.scatter(xone[i], xtwo[i], marker = 'x',color = 'g')


    plot_line(X,y,sigma,mu,phi,dataset,ax1)
    fig1.suptitle('Scatter Plot and decision boundary')
    plt.title(titl)
    blackline = mlines.Line2D([], [], color = 'black')
    fig1.legend([clas1,clas2,clas3,blackline], ['class1', 'class2', 'class3' ,'Linear Boundary'],loc = 2)
    fig1.legend([mis1,mis2,mis3], ['mis1', 'mis2', 'mis3'],loc = 1)


def plot_line(X,y,sigma,mu,phi,dataset,plot):
    color = ['blue','red','green']

    for i in range(3):
        if(dataset =='1'):
            if(i==0):
                j=i+1
                x, y = np.meshgrid(np.linspace(-1, 4.9), np.linspace(-1, 5.45))
            if(i==1):
                j=i+1
                x, y = np.meshgrid(np.linspace(4.5, 12), np.linspace(4,10))
            if(i==2):
                j=0
                x, y = np.meshgrid(np.linspace(4.4, 12), np.linspace(5.45, 10))

        if(dataset =='2'):
            if(i==0):
                j=i+1
                x, y = np.meshgrid(np.linspace(-1,4.7), np.linspace(-1, 5.8))
            if(i==1):
                j=i+1
                x, y = np.meshgrid(np.linspace(4.7, 12), np.linspace(-1,10))
            if(i==2):
                j=0
                x, y = np.meshgrid(np.linspace(-1, 5.3), np.linspace(5.6, 10))

        pi1 = phi[i]
        pi0 = phi[j]
        mu_1 = mu[i]
        mu_0 =mu[j]
        Sigma_1 = sigma[i]
        Sigma_0 = sigma[j]
        invsigma1 = np.linalg.inv(Sigma_1)
        invsigma0 = np.linalg.inv(Sigma_0)
        c = -(np.dot(mu_1.T, np.dot(invsigma1, mu_1)) - np.dot(mu_0.T, np.dot(invsigma0, mu_0)))/2 + np.log(pi0/(pi1)) + np.log(np.linalg.det(invsigma1)/np.linalg.det(invsigma0))/2
        b = np.dot(invsigma1, mu_1) - np.dot(invsigma0, mu_0)
        a = (invsigma0 - invsigma1)/2
        z = c + b[0]*x + b[1]*y + a[0, 0]*(x**2) + (a[0, 1] + a[1, 0])*x*y + a[1, 1]*(y**2)
        plot.contour(x, y, z,0,colors='black')
